In [2]:
import ee
import geemap


ee.Authenticate()
ee.Initialize(project='raman-461708')
# Initialize the Earth Engine API (run ee.Authenticate() once before this in a new environment)

roi = ee.FeatureCollection("users/mtpictd/agro_eco_regions")#.filter(ee.Filter.neq('ae_regcode', 2))
agrifield_data = ee.FeatureCollection("projects/raman-461708/assets/agrifieldnet_all")
#agrifield_data = ee.FeatureCollection("projects/raman-461708/assets/india_10k_fields")


# Create a geemap map
m = geemap.Map(center=[22, 80], zoom=4)


url = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
m.layout.height = '1000px'
m.add_tile_layer(url, name="Google Map", attribution="Google")

m.addLayer(agrifield_data, {}, "Agri-Field")

# Display the map (in a Jupyter notebook)
m

Map(center=[22, 80], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tran…

In [2]:
def get_lulc_test_band_ours_band(exp):
    if exp == "OURS":
        lulc_ours = ee.Image(
        "projects/raman-461708/assets/LULC_v4_PanIndia_2020-07-01_2021-06-30"
        )
        # CHANGE THESE to the actual band names in your assets
        
        OURS_BAND = 'predicted_label'          # band in lulc_ours with classes 8–12
        FARM_LIST = [8,9,10,11, 13]
        return OURS_BAND, lulc_ours, FARM_LIST
    
    elif exp == "WC":
        # -------------------------------------------------------------
        # 1. INPUTS
        # -------------------------------------------------------------

 


        lulc_ours = ee.ImageCollection('ESA/WorldCover/v200').first().clip(roi)
        OURS_BAND = 'Map'
        FARM_LIST = [40]
        return OURS_BAND, lulc_ours, FARM_LIST
    
    elif exp == "ESRI":
        # ESRI LULC collection
        COLLECTION_ID = "projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS"

        # Original → remapped classes (same as your JS CONFIG)
        REMAP_FROM = [1, 2, 4, 5, 7, 8, 9, 10, 11]
        REMAP_TO   = [1, 2, 3, 4, 5, 6, 7, 8, 9]
        # After remap:
        # 4 = Crops
        # 9 = Rangeland

        esri_lulc = ee.ImageCollection(COLLECTION_ID)

        def get_esri_lulc_year(year):
            """Return remapped ESRI LULC composite for a given year."""
            start = ee.Date.fromYMD(year, 1, 1)
            end   = ee.Date.fromYMD(year, 12, 31)

            img = esri_lulc.filterDate(start, end).mosaic()
            # Remap first band; name it 'lulc'
            remapped = img.remap(REMAP_FROM, REMAP_TO, 0)
            return remapped.rename('lulc')

        # Example: get 2020 map
        year = 2022
        lulc_ours = get_esri_lulc_year(year)
        OURS_BAND = 'lulc'
        FARM_LIST = [4]
        return OURS_BAND, lulc_ours, FARM_LIST
    elif exp == "DW_O":
        lulc_ours = ic.select("b1").mosaic().clip(roi)
        OURS_BAND = 'b1'
        FARM_LIST = [7,12]

        return OURS_BAND, lulc_ours, FARM_LIST
    elif exp == "DW_M":
        lulc_ours = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").filterDate('2023-01-01', '2023-12-31').mode().clip(roi)
        OURS_BAND = 'label'
        FARM_LIST = [4]

        return OURS_BAND, lulc_ours, FARM_LIST
    elif exp == "WRI":
        lulc_ours = ee.ImageCollection("projects/global-pasture-watch/assets/ggc-30m/v1/grassland_c").mosaic().clip(roi)
        OURS_BAND = "dominant_class"
        FARM_LIST = [2]
        return OURS_BAND, lulc_ours, FARM_LIST
        
TODO = "OURS"    
OURS_BAND, lulc_ours, FARM_LIST = get_lulc_test_band_ours_band(TODO)


In [ ]:

lulc_clip = lulc_ours.select(OURS_BAND)
farm_binary = lulc_clip.remap(
    FARM_LIST,                # from these classes
    [1] * len(FARM_LIST),     # to 1 (farm)
    0                            # everything else -> 0 (rest)
).rename('farm')

rest_binary = lulc_clip.remap(
    FARM_LIST,                # from these classes
    [0] * len(FARM_LIST),     # to 1 (farm)
    1                            # everything else -> 0 (rest)
).rename('rest')

sum_dict = farm_binary.reduceRegions(
    reducer=ee.Reducer.sum(),
    collection=agrifield_data,
    scale=10,
    tileScale=4               # important for big regions
)

count_dict = rest_binary.reduceRegions(
    reducer=ee.Reducer.sum(),
    collection=agrifield_data,
    scale=10,
    tileScale=4
)



In [4]:
total_farm = sum_dict.reduceColumns(
    reducer=ee.Reducer.sum(),
    selectors=['sum']
).get('sum')


In [5]:
total_rest = count_dict.reduceColumns(
    reducer=ee.Reducer.sum(),
    selectors=['sum']
).get('sum')


In [ ]:
total_farm.getInfo()/(total_farm.getInfo()+total_rest.getInfo())*100

In [53]:
total_farm

In [8]:
VIS_PALETTE = [
    '419bdf',
    '397d49',
    '88b053',
    '7a87c6',
    'e49635',
    'dfc35a',
    'c4281b',
    'a59b8f',
    'b39fe1',
]

palette_corrected = ['000000','ff0000','74ccf4','1ca3ec','0f5e9c','f1c232','38761d', 'A9A9A9', 'BAD93E', 'f59d22','FF9371','b3561d', 'a9a9a9', '84994F']

m.addLayer(lulc_ours.clip(roi).select(OURS_BAND), {'min':0, 'max':8, 'palette':palette_corrected}, "LULC")
m.addLayer(agrifield_data, {}, "Agri-Field")
m

Map(bottom=2291.0, center=[22, 80], controls=(WidgetControl(options=['position', 'transparent_bg'], position='…

In [20]:
agrifield_data.size()

In [53]:
roi.limit(1)

In [37]:
import ee

ee.Authenticate()
ee.Initialize(project='raman-461708')
roi = ee.FeatureCollection("users/mtpictd/agro_eco_regions")
#fields = ee.FeatureCollection("projects/raman-461708/assets/india_10k_fields")  # or your existing variable
fields = agrifield_data
k = 5      # number of random points per field
seed = 42  # for reproducibility

def sample_field(f):
    pts = ee.FeatureCollection.randomPoints(
        region = f.geometry(),
        points = k,
        seed = seed
    ).map(lambda p: p.copyProperties(f))  # copy field properties onto each point
    return pts

# Map over all fields and flatten to a single FeatureCollection
points_fc = fields.map(sample_field).flatten()

In [19]:
task = ee.batch.Export.table.toAsset(
    collection = points_fc,
    description = 'export_field_points',
    assetId = 'projects/raman-461708/assets/10kfield_points'
)

task.start()

In [33]:
def get_lulc_test_band_ours_band(exp):
    if exp == "OURS":
        lulc_ours = ee.Image(
        "projects/raman-461708/assets/LULC_v4_PanIndia_2020-07-01_2021-06-30"
        )
        # CHANGE THESE to the actual band names in your assets
        
        OURS_BAND = 'predicted_label'          # band in lulc_ours with classes 8–12
        FARM_LIST = [8,9,10,11, 13]
        return OURS_BAND, lulc_ours, FARM_LIST
    
    elif exp == "WC":
        # -------------------------------------------------------------
        # 1. INPUTS
        # -------------------------------------------------------------

 


        lulc_ours = ee.ImageCollection('ESA/WorldCover/v200').first().clip(roi)
        OURS_BAND = 'Map'
        FARM_LIST = [40]
        return OURS_BAND, lulc_ours, FARM_LIST
    
    elif exp == "ESRI":
        # ESRI LULC collection
        COLLECTION_ID = "projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS"

        # Original → remapped classes (same as your JS CONFIG)
        REMAP_FROM = [1, 2, 4, 5, 7, 8, 9, 10, 11]
        REMAP_TO   = [1, 2, 3, 4, 5, 6, 7, 8, 9]
        # After remap:
        # 4 = Crops
        # 9 = Rangeland

        esri_lulc = ee.ImageCollection(COLLECTION_ID)

        def get_esri_lulc_year(year):
            """Return remapped ESRI LULC composite for a given year."""
            start = ee.Date.fromYMD(year, 1, 1)
            end   = ee.Date.fromYMD(year, 12, 31)

            img = esri_lulc.filterDate(start, end).mosaic()
            # Remap first band; name it 'lulc'
            remapped = img.remap(REMAP_FROM, REMAP_TO, 0)
            return remapped.rename('lulc')

        # Example: get 2020 map
        year = 2022
        lulc_ours = get_esri_lulc_year(year)
        OURS_BAND = 'lulc'
        FARM_LIST = [4]
        return OURS_BAND, lulc_ours, FARM_LIST
    elif exp == "DW_O":
        lulc_ours = ic.select("b1").mosaic().clip(roi)
        OURS_BAND = 'b1'
        FARM_LIST = [7,12]

        return OURS_BAND, lulc_ours, FARM_LIST
    elif exp == "DW_M":
        lulc_ours = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").filterDate('2023-01-01', '2023-12-31').mode().clip(roi)
        OURS_BAND = 'label'
        FARM_LIST = [4]

        return OURS_BAND, lulc_ours, FARM_LIST
    elif exp == "WRI":
        lulc_ours = ee.ImageCollection("projects/global-pasture-watch/assets/ggc-30m/v1/grassland_c").mosaic().clip(roi)
        OURS_BAND = "dominant_class"
        FARM_LIST = [2]
        return OURS_BAND, lulc_ours, FARM_LIST
        
TODO = "ESRI"    
OURS_BAND, lulc_ours, FARM_LIST = get_lulc_test_band_ours_band(TODO)


In [34]:
points_fc = ee.FeatureCollection("projects/raman-461708/assets/10kfield_points")



lulc_clip = lulc_ours.select(OURS_BAND)
farm_binary = lulc_clip.remap(
    FARM_LIST,                # from these classes
    [1] * len(FARM_LIST),     # to 1 (farm)
    0                            # everything else -> 0 (rest)
).rename('farm')

rest_binary = lulc_clip.remap(
    FARM_LIST,                # from these classes
    [0] * len(FARM_LIST),     # to 1 (farm)
    1                            # everything else -> 0 (rest)
).rename('rest')

sum_dict = farm_binary.reduceRegions(
    reducer=ee.Reducer.sum(),
    collection=points_fc,
    scale=10,
    tileScale=4               # important for big regions
)

count_dict = rest_binary.reduceRegions(
    reducer=ee.Reducer.sum(),
    collection=points_fc,
    scale=10,
    tileScale=4
)



In [35]:
total_farm = sum_dict.reduceColumns(
    reducer=ee.Reducer.sum(),
    selectors=['sum']
).get('sum')
total_rest = count_dict.reduceColumns(
    reducer=ee.Reducer.sum(),
    selectors=['sum']
).get('sum')



In [36]:
total_farm.getInfo()/(total_farm.getInfo()+total_rest.getInfo())*100

85.37301508039549

In [28]:
total_farm.getInfo()

45040.337254901955

In [9]:
sampled.limit(1)

In [10]:
OURS_BAND

'predicted_label'